# Sandpiper usage

Sandpiper is a WPS server that resolves rules for climatological impacts. To get started, first instatiate the client. Here, the client will try to connect to a remote `sandpiper` instance using the `url` parameter.

In [13]:
from birdy import WPSClient
import os
from wps_tools.testing import get_target_url
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "sandpiper":
    os.chdir('../')

In [14]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("sandpiper")
print(f"Using sandpiper on {url}")

Using sandpiper on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/sandpiper/wps


In [15]:
sandpiper = WPSClient(url)

The list of available processes can be diplayed using the `?` command (ex. `bird?`). Furthermore, help for individual processes can be diplayed using the same command (ex. `bird.process?`).

In [16]:
# NBVAL_IGNORE_OUTPUT
sandpiper?
sandpiper.resolve_rules?

Signature:
sandpiper.resolve_rules(
    csv=None,
    date_range='2080',
    region='bc',
    geoserver='https://docker-dev03.pcic.uvic.ca/geoserver/bc_regions/ows',
    ensemble='p2a_rules',
    loglevel='INFO',
    connection_string=None,
    thredds=True,
    output_formats=None,
)
Docstring:
Resolve climatological impacts rules

Parameters
----------
csv : ComplexData:mimetype:`text/csv`, :mimetype:`text/plain`
    A CSV document
date_range : {'2020', '2050', '2080'}string
    30 year period for data
region : {'bc', 'alberni_clayoquot', 'boreal_plains', 'bulkley_nechako', 'capital', 'cariboo', 'central_coast', 'central_kootenay', 'central_okanagan', 'columbia_shuswap', ...}string
    Impacted region
geoserver : string
    Geoserver URL
connection_string : string
    Database connection string
ensemble : string
    Ensemble name filter for data files
thredds : boolean
    Data from thredds server. It is not recommended to change from the default (True)
loglevel : {'CRITICAL', 'ERROR

Now that we know which process we wish to run, we can use the docstring to ensure we provide the appropriate parameters. The main input is the `csv` file containing all the rules. You can either pass it raw output from github, or pass a local file. The contents are read as a stream meaning you will have to do a little processing first with a local file.

In [17]:
output_online = sandpiper.resolve_rules(
    csv = 'https://raw.githubusercontent.com/pacificclimate/sandpiper/master/tests/data/rules_small.csv'   
)

In [18]:
output_local = sandpiper.resolve_rules(
    csv = open(resource_filename('tests', 'data/rules_small.csv')).read()
)

In [19]:
# Access the json contents as a dictionary by setting asobj=True
rules_online = output_online.get(asobj=True)[0]
rules_local = output_local.get(asobj=True)[0]

Once the process has completed we can extract the results and ensure it is what we expected.

In [20]:
expected_rules = {
    'rule_snow': True,
    'rule_hybrid': True,
    'rule_rain': True
}

assert rules_online == expected_rules
assert rules_local == expected_rules